In [89]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report, average_precision_score, precision_recall_curve




In [90]:
# 1. 데이터 로드
df = pd.read_csv(r'C:\Users\pc57\Downloads\creditcard\creditcard.csv')
print("데이터 로드 성공")

데이터 로드 성공


In [91]:
# 2. 샘플링
# 명세: 사기(1)는 유지, 정상(0)은 10,000개만 랜덤 샘플링
fraud = df[df['Class'] == 1]
normal = df[df['Class'] == 0]

# 정상 데이터 10,000개 추출 (random_state=42)
normal_sample = normal.sample(n=10000, random_state=42)

# 두 데이터 합치기
df_sampled = pd.concat([fraud, normal_sample])

# 데이터 불균형 확인
print("\n[2] 샘플링 후 데이터 개수:", len(df_sampled))
print(df_sampled['Class'].value_counts())


[2] 샘플링 후 데이터 개수: 10492
Class
0    10000
1      492
Name: count, dtype: int64


In [92]:

# 3. 데이터 전처리 (Amount 표준화)
scaler = StandardScaler()
df_sampled['Amount_Scaled'] = scaler.fit_transform(df_sampled['Amount'].values.reshape(-1, 1))
df_sampled = df_sampled.drop(['Amount', 'Time'], axis=1)

X = df_sampled.drop('Class', axis=1)
y = df_sampled['Class']


In [93]:
# 4. 학습/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print("\n[4] 분할 후 학습 데이터 비율:")
print(y_train.value_counts(normalize=True))


[4] 분할 후 학습 데이터 비율:
Class
0    0.953056
1    0.046944
Name: proportion, dtype: float64


In [94]:
# 5. SMOTE 적용
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print("\n[5] SMOTE 적용 후 데이터 개수:")
print(pd.Series(y_train_res).value_counts().to_dict())


[5] SMOTE 적용 후 데이터 개수:
{0: 7999, 1: 7999}


In [95]:
# 지표 계산 메소드
def get_best_threshold(model, X_test, y_test, name):
    '''
    데이터 샘플링(Under-sampling)과 SMOTE(Over-sampling)를 통해 클래스 불균형을 완화한 후, 
    성능을 극대화하기 위한 최적화(Optimization) 전략

    임계값 조정 (Threshold Tuning)
    Precision-Recall Curve를 분석하여 F1-score가 최대가 되는 최적의 임계값을 찾아냄
    <- 학습 데이터는 SMOTE를 통해 1:1 비율이 되었으나, 테스트 데이터는 여전히 불균형(Imbalanced) 상태이기 때문
    -> Precision(정밀도)와 Recall(재현율)의 트레이드오프(Trade-off) 관계를 고려

    이후...+ 하이퍼파라미터 튜닝 (Criterion & Estimators) 
    불순도 지표를 Gini에서 Entropy로 변경하고, 트리의 개수(n_estimators)를 100개에서 300~500개로 늘람
    -> Entropy는 클래스 불순도에 더 민감하게 반응
    -> 트리 개수 증가는 과적합을 방지하고 일반화 성능을 높이는 데 기여

    ++ 모델 고도화
    최종 모델로 RandomForestClassifier 대신 ExtraTreesClassifier (Extremely Randomized Trees)를 선정
    -> 변동성 감소, SMOTE 노이즈에 대한 강건성
    '''
    probs = model.predict_proba(X_test)[:, 1]
    precisions, recalls, thresholds = precision_recall_curve(y_test, probs)
    f1s = 2 * (precisions * recalls) / (precisions + recalls + 1e-10)
    best_idx = np.argmax(f1s)
    
    # PR-AUC
    pr_auc = average_precision_score(y_test, probs)
    
    print(f"\n>> [{name}] 성적표")
    print(f"   PR-AUC Score: {pr_auc:.4f}")
    print(f"   Best Threshold: {thresholds[best_idx]:.4f}")
    print(f"   예상 F1: {f1s[best_idx]:.4f} (Precision: {precisions[best_idx]:.4f}, Recall: {recalls[best_idx]:.4f})")
    
    # 리포트 출력용
    preds = (probs >= thresholds[best_idx]).astype(int)
    print(classification_report(y_test, preds))
    return pr_auc

In [96]:
et_final = ExtraTreesClassifier(
    n_estimators=500, 
    criterion='entropy', 
    max_depth=None, 
    min_samples_split=2, 
    random_state=42, 
    n_jobs=-1
)

print("모델 학습 중...")
et_final.fit(X_train_res, y_train_res)
print("학습 완료!")

# 결과 출력 함수 호출 (이름도 ExtraTrees로 변경)
get_best_threshold(et_final, X_test, y_test, "ExtraTrees 하이퍼파라미터 + Threshold 조정")

모델 학습 중...
학습 완료!

>> [ExtraTrees 하이퍼파라미터 + Threshold 조정] 성적표
   PR-AUC Score: 0.9529
   Best Threshold: 0.4920
   예상 F1: 0.9305 (Precision: 0.9775, Recall: 0.8878)
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      2001
           1       0.98      0.89      0.93        98

    accuracy                           0.99      2099
   macro avg       0.99      0.94      0.96      2099
weighted avg       0.99      0.99      0.99      2099



0.9529187228152467